In [1]:
import xlrd
import xlwt

import random

from math import floor

In [2]:
class Student():
    
    def __init__(self, sciper, sex, name):
        self.data = {    
            'SCIPER' : int(sciper),
            'sex' : sex,
            'name' : name,
            'group' : 0,
            'nationality' : "none",
            'phone' : "none",
            'facebook_invited' : False,
            'facebook_member' : False
        }
    
    def show(self):
        print(str(self.data['SCIPER'])+"; "+self.data['sex']+"; "+self.data['name'])

In [3]:
def open_workbook():
    
    filename_valid = False
    while not filename_valid:
        filename = input("Enter filename : ")
        filename_valid = True
        try:
            workbook = xlrd.open_workbook(filename)
        except:
            filename_valid = False
            print("Unable to open file...")
    
    print("\n Opened " + str(filename))
    
    # check the import parameters
    
    print("\nThis workbook contains the following sheets:")
    print(workbook.sheet_names())
    sheet_index = int(input("Which sheet contains the data? : "))
    
    sheet = workbook.sheet_by_index(sheet_index)
    
    print("\nThese are the first 5 rows beginnings of the sheet : \n")
    for i in range(0,5):
        line = "Line " + str(i) + " : "
        for j in range(0,3):
            line += str(sheet.cell(i,j).value) + "; "
        print(line)
    row = int(input("Which row contains the column headers? : "))
    
    print("\nThese are the headers:")
    for i in range(0,sheet.ncols):
        print(str(i) + " : " + str(sheet.cell(row,i).value))
    
    return sheet, row

In [20]:
def import_column(students):
        
    sheet, header_row = open_workbook()
    
    sample = Student(12324,"M","John Smith")
    
    print("\nYou can add one of the following keys:")
    for k in sample.data.keys():
        print(k)
    
    
    valid_key = False
    while not valid_key:
        key = input("Which key do you want to add to ? : ")
        if key in sample.data.keys():
            valid_key = True
        else:
            print("Key invalid")
    
    print("\nWhich column contains the :")
    sciper = int(input("SCIPER : "))
    column = int(input("your data : "))
    
    for i in range(header_row+1,sheet.nrows):
        row = sheet.row(i)
        for student in students:
            if student.data['SCIPER'] == row[sciper].value:
                student.data[key] = row[column].value
    
    return students
    

In [5]:
def import_check_headers():

    sheet, header_row = open_workbook()
    
    print("\nWhich column contains the :")
    name = int(input("Name : "))
    sciper = int(input("SCIPER : "))
    sex = int(input("Sex : "))
    
    # start importing
    
    students = []
    for i in range(header_row+1,sheet.nrows):
        row = sheet.row(i)
        
        # change empty SCIPER fields to 0
        if (row[sciper].value == 'No Sciper'):
            row[sciper].value = 0
        
        # infer sex from saluation
        if (row[sex].value == "Madame"):
            row[sex].value = "F"
        elif (row[sex].value == "Monsieur"):
            row[sex].value = "M"
        
        students.append(Student(row[sciper].value,row[sex].value,row[name].value))
    return students

In [6]:
def write_to_file(student_list,filename):
    workbook = xlwt.Workbook()
    sheet = workbook.add_sheet("Sheet1")
    
    j = int(0)
    for k in students[0].data.keys():
        sheet.write(0,j,k)
        j += 1
    
    i = int(1)
    for student in student_list:
        j = int(0)
        for k in student.data.keys(): 
            sheet.write(i,j,student.data[k])
            j += 1
        i += 1
    
    workbook.save(filename)

In [7]:
"""
This function will merge an old and new list of students, while
keeping info from the old data.
The merge is based in SCIPER numbering
"""
def merge_students(old_list,new_list):
    merged_list = []
    for new_student in new_list:
        existed = False
        for old_student in old_list:
            if new_student.data['SCIPER'] == old_student.data['SCIPER']:
                existed = True
                merged_list.append(old_student)
                break
        if existed == False:
            merged_list.append(new_student)
    return merged_list

def find_new_students(old_list,new_list):
    new_students = []
    for new_student in new_list:
        existed = False
        for old_student in old_list:
            if new_student.data['SCIPER'] == old_student.data['SCIPER']:
                existed = True
                break
        if existed == False:
            new_students.append(new_student)
    
    return new_students

In [8]:
def assign_students(students,num_groups):
    
    # find required group size
    unassigned = get_students_by_group(students,0)
    group_size = floor(len(unassigned) / (num_groups - 1))
    
    print("Distributing " + str(len(unassigned))+" into "+str(num_groups))
    print("Using a group size of " + str(group_size))
    
    # assign students to each group
    for i in range(1,num_groups):
        placed = 0
        while (placed < group_size):
            j = random.randrange(0,len(unassigned))
            if students[j].data['group'] == 0:
                students[j].data['group'] = i
                placed += 1
    
    # assign the students that have been left out
    placed_positions = []
    for student in students:
        if student.data['group'] == 0:
            placed = False
            while not placed:
                i = random.randrange(1,num_groups)
                
                # make sure not to assign two overflow students
                # to the same group, since they can all fit once
                
                if i not in placed_positions:
                    placed_positions.append(i)
                    student.data['group'] = i
                    placed = True
                
        
    return students

def get_students_by_group(students,group):
    current = []
    for student in students:
        if student.data['group'] == group:
            current.append(student)
    return current


In [9]:
def show_group_stats(students,group_num):
    group = get_students_by_group(students,group_num)
    
    print("\nStats for group " + str(group_num))
    print("Number of students: " + str(len(group)))
    m = float(0)
    for student in group:
        if student.data['sex'] == "M":
            m = m+1
    part_male = m / float(len(group))
    print("Percentage male: " + str(part_male))


In [10]:
students = import_check_headers()

Enter filename : new_assigned.xlsx

 Opened new_assigned.xlsx

This workbook contains the following sheets:
['Sheet1']
Which sheet contains the data? : 0

These are the first 5 rows beginnings of the sheet : 

Line 0 : SCIPER; sex; phone; 
Line 1 : 247331.0; F; none; 
Line 2 : 262217.0; M; none; 
Line 3 : 261343.0; F; none; 
Line 4 : 245062.0; M; none; 
Which row contains the column headers? : 0

These are the headers:
0 : SCIPER
1 : sex
2 : phone
3 : group
4 : facebook_member
5 : facebook_invited
6 : name
7 : nationality

Which column contains the :
Name : 6
SCIPER : 0
Sex : 1


In [11]:
students.append(Student(13432,"M","Test Testy"))

In [12]:
for student in students:
    student.show()

247331; F; Abdennadher Amna
262217; M; Aebi Benjamin Giovanni Numa
261343; F; Ait Oumelloul Mariam
245062; M; Alexandre Jérémy Constant Henry
263918; M; Ancillon Jules Edouard Pierre
251773; F; Angles Mathilde Marie Véronique
262114; F; Artaz Inès Kim-Maï Claude
249684; M; Babey Arthur Léo Maurin
259150; F; Baleydier Inès Orphée Aster
259300; F; Ballabio Michelle Giada
259168; F; Barbier Emma Jeanne Bénédicte
246347; M; Barbier De Preville Melchior Loïc Marie
259331; M; Barret Lucien William Robin
259611; F; Bédat Morane Marie Philippine
198199; M; Bekri Ibrahim
251772; F; Ben Mrad Ines
261675; M; Ben Romdhane Ahmed
250150; F; Benedetti Manon
257114; M; Bérard Simon
247179; M; Berger Julien Daniel
259003; M; Berger Thomas Bernard
262178; M; Bergkvist Eric Hans Ildo
246348; M; Bernaert Vincent Arnaud
257360; F; Besson Julia Aleida Jeannette
258782; M; Bettoni Guillaume Ludovic
245915; M; Bian Theophile Léonard
263815; M; Bichon Nicolas Frederic Augustin
248142; M; Bigot Valentin Pierre


In [17]:
students = assign_students(students,17)

Distributing 0 into 17
Using a group size of 0


In [21]:
students = import_column(students)

Enter filename : assigned.xlsx

 Opened assigned.xlsx

This workbook contains the following sheets:
['Sheet1']
Which sheet contains the data? : 0

These are the first 5 rows beginnings of the sheet : 

Line 0 : SCIPER; Sex; Group coaching; 
Line 1 : 247331.0; F; 8.0; 
Line 2 : 262217.0; M; 1.0; 
Line 3 : 261343.0; F; 3.0; 
Line 4 : 245062.0; M; 12.0; 
Which row contains the column headers? : 0

These are the headers:
0 : SCIPER
1 : Sex
2 : Group coaching
3 : Name
4 : email

You can add one of the following keys:
sex
nationality
name
group
facebook_member
phone
facebook_invited
SCIPER
Which key do you want to add to ? : group

Which column contains the :
SCIPER : 0
your data : 2


In [22]:
write_to_file(students,"combined.xlsx")